In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [2]:
#importing files
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('SampleSubmission.csv')

In [3]:
def one_hot_encoding(traindata, testdata, *args):
    
    for ii in args:
        traindata = pd.get_dummies(traindata, prefix=[ii], columns=[ii])
        testdata = pd.get_dummies(testdata, prefix=[ii], columns=[ii])
        
    return traindata, testdata

def drop_columns(traindata, testdata, target, *args):
    
    labels = np.array(traindata[target])
    
    columns = []
    for _ in args:
        columns.append(_)
        
    traindata = traindata.drop(columns, axis=1)
    #traindata = traindata.drop(target, axis=1)
    testdata = testdata.drop(columns, axis=1)
        
    return labels, traindata, testdata

In [4]:
def fill_missing_values(data):
    
    '''
    Function to input missing values based on the column object type
    '''
    
    cols = list(data.columns)
    for col in cols:
        if data[col].dtype == 'int64' or data[col].dtype == 'float64':
        
            data[col] = data[col].fillna(data[col].mean())
        
        #elif data[col].dtype == 'O' or data[col].dtype == 'object':
        #    data[col] = data[col].fillna(data[col].mode()[0])
            
        else:
            data[col] = data[col].fillna(data[col].mode()[0])
            
    return data

In [5]:
def process(traindata, testdata):
    
    cols = list(traindata.columns)
    for _ in cols:
        traindata[_] = np.where(traindata[_] == np.inf, 0, traindata[_])
        testdata[_] = np.where(testdata[_] == np.inf, 0, testdata[_])
        traindata[_] = np.where(traindata[_] == np.nan, 0, traindata[_])
        testdata[_] = np.where(testdata[_] == np.nan, 0, testdata[_])
        traindata[_] = np.where(traindata[_] == -np.inf, 0, traindata[_])
        testdata[_] = np.where(testdata[_] == -np.inf, 0, testdata[_])
        
    return traindata, testdata

In [37]:
def freq_encode(data, cols):
    for i in cols:
        encoding = data.groupby(i).size()
        encoding = encoding/len(data)
        data[i + '_enc'] = data[i].map(encoding)
    return data

def mean_target(data, cols):
    kf = KFold(5)
    a = pd.DataFrame()
    for tr_ind, val_ind in kf.split(data):
        X_tr, X_val= data.iloc[tr_ind].copy(), data.iloc[val_ind].copy()
        for col in cols:
            means = X_val[col].map(X_tr.groupby(col).Item_Store_Returns.mean())
            X_val[col + '_mean_target'] = means + 0.0001
        a = pd.concat((a, X_val))
    prior = target.mean()
    a.fillna(prior, inplace=True)
    return a

In [7]:
ntrain = train.shape[0]
ntest = test.shape[0]
target = train.Item_Store_Returns.copy()
df = pd.concat((train, test)).reset_index(drop=True)

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990 entries, 0 to 4989
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Item_ID              4990 non-null   object 
 1   Store_ID             4990 non-null   object 
 2   Item_Store_ID        4990 non-null   object 
 3   Item_Weight          4188 non-null   float64
 4   Item_Sugar_Content   4990 non-null   object 
 5   Item_Visibility      4990 non-null   float64
 6   Item_Type            4990 non-null   object 
 7   Item_Price           4990 non-null   float64
 8   Store_Start_Year     4990 non-null   int64  
 9   Store_Size           3540 non-null   object 
 10  Store_Location_Type  4990 non-null   object 
 11  Store_Type           4990 non-null   object 
 12  Item_Store_Returns   4990 non-null   float64
dtypes: float64(4), int64(1), object(8)
memory usage: 506.9+ KB


In [45]:
train.Item_ID.value_counts()

FDG33    9
NCL31    9
DRE49    9
FDN56    8
FDU13    8
        ..
FDS13    1
FDX44    1
FDO23    1
FDH16    1
FDL21    1
Name: Item_ID, Length: 1451, dtype: int64

In [27]:
drop_col = ['Item_Store_ID']
cat_cols = ['Store_ID', 'Item_Sugar_Content', 'Item_Type', 'Store_Size', 'Store_Location_Type', 'Store_Type']
num_cols = ['Item_Weight', 'Item_Visibility', 'Item_Price', 'Store_Start_Year']

labels, train1, test1 = drop_columns(df, test, 'Item_Store_Returns', *drop_col)

one_hot_cols = ['REGION', 'TENURE']

#df, test1 = one_hot_encoding(df, test, *one_hot_cols)

In [32]:
df.shape

(8522, 12)

In [31]:
df = df.drop('Item_Store_ID', axis=1)

In [40]:
df = freq_encode(df, cat_cols)
data = df.copy()

In [41]:
data.shape

(8522, 18)

In [50]:
data.head()

,Item_ID,Store_ID,Item_Weight,Item_Sugar_Content,Item_Visibility,Item_Type,Item_Price,Store_Start_Year,Store_Size,Store_Location_Type,...,Store_Location_Type_enc,Store_Type_enc,Store_ID_mean_target,Item_Sugar_Content_mean_target,Item_Type_mean_target,Store_Size_mean_target,Store_Location_Type_mean_target,Store_Type_mean_target,Item_ID_mean_target,Item_ID_enc
0,DRA12,BABATUNJI010,11.6,Low Sugar,0.068535,Soft Drinks,357.54,2005,6103.52,Cluster 3,...,0.392983,0.126966,638.542619,5998.375729,6103.520164,6103.520164,5783.390945,844.827098,6103.520164,0.000704
1,DRA12,BABATUNJI013,11.6,Low Sugar,0.040912,Soft Drinks,355.79,1994,High,Cluster 3,...,0.392983,0.654424,4050.556941,5998.375729,6103.520164,4050.556941,5783.390945,6402.821654,6103.520164,0.000704
2,DRA12,BABATUNJI017,11.6,Low Sugar,0.041178,Soft Drinks,350.79,2014,6103.52,Cluster 2,...,0.326801,0.654424,7558.126740,5998.375729,6103.520164,6103.520164,7579.261892,6402.821654,6103.520164,0.000704
3,DRA12,BABATUNJI018,11.6,Low Sugar,0.041113,Soft Drinks,355.04,2016,Medium,Cluster 3,...,0.392983,0.108895,5565.345977,5998.375729,6103.520164,7974.965018,5783.390945,5565.345977,6103.520164,0.000704
4,DRA12,BABATUNJI035,11.6,Ultra Low Sugar,0.000000,Soft Drinks,354.79,2011,Small,Cluster 2,...,0.326801,0.654424,7886.198484,5682.390784,6103.520164,5304.745551,7579.261892,6402.821654,6103.520164,0.000704


In [43]:
data = mean_target(data, cat_cols)

In [49]:
data.shape

(8522, 26)

In [48]:
data = freq_encode(data, ['Item_ID'])

In [51]:
data, test1 = one_hot_encoding(data, test, *cat_cols)

In [52]:
data.shape

(8522, 60)

In [53]:
data.head()

,Item_ID,Item_Weight,Item_Visibility,Item_Price,Store_Start_Year,Item_Store_Returns,Store_ID_enc,Item_Sugar_Content_enc,Item_Type_enc,Store_Size_enc,...,Store_Size_High,Store_Size_Medium,Store_Size_Small,Store_Location_Type_Cluster 1,Store_Location_Type_Cluster 2,Store_Location_Type_Cluster 3,Store_Type_Grocery Store,Store_Type_Supermarket Type1,Store_Type_Supermarket Type2,Store_Type_Supermarket Type3
0,DRA12,11.6,0.068535,357.54,2005,709.08,0.065008,0.610185,0.052218,6103.520164,...,0,0,0,0,0,1,1,0,0,0
1,DRA12,11.6,0.040912,355.79,1994,6381.69,0.109364,0.610185,0.052218,0.109364,...,1,0,0,0,0,1,0,1,0,0
2,DRA12,11.6,0.041178,350.79,2014,6381.69,0.108660,0.610185,0.052218,6103.520164,...,0,0,0,0,1,0,0,1,0,0
3,DRA12,11.6,0.041113,355.04,2016,2127.23,0.108895,0.610185,0.052218,0.327740,...,0,1,0,0,0,1,0,0,1,0
4,DRA12,11.6,0.000000,354.79,2011,2481.77,0.109129,0.037080,0.052218,0.280216,...,0,0,1,0,1,0,0,1,0,0


In [55]:
data = data.drop('Item_ID', axis=1)

In [56]:
data.shape

(8522, 59)

In [59]:
train2 = data[:ntrain].copy()
target = train2.Item_Store_Returns.copy()
train2.drop('Item_Store_Returns', axis=1, inplace=True)

test2 = data[ntrain:].copy()
test2.drop('Item_Store_Returns', axis=1, inplace=True)
test2 = test2.reset_index(drop=True)

In [68]:
train2.shape, test2.shape

((4990, 58), (3532, 58))

In [61]:
traindata = train2.copy()
testdata = test2.copy()

In [62]:
traindata = traindata.fillna(-999)
testdata = testdata.fillna(-999)

In [63]:
#using StandardScaler function to scale the numeric features 

scaler = preprocessing.StandardScaler().fit(traindata)
traindata = scaler.transform(traindata)
traindata

array([[-0.43819088,  0.03051274, -0.28704749, ..., -1.39988137,
        -0.33370432, -0.31942417],
       [-0.43819088, -0.49015852, -0.30170824, ...,  0.71434624,
        -0.33370432, -0.31942417],
       [-0.43819088, -0.48515069, -0.34359611, ...,  0.71434624,
        -0.33370432, -0.31942417],
       ...,
       [-0.4368274 ,  0.31312417,  0.12010262, ...,  0.71434624,
        -0.33370432, -0.31942417],
       [-0.4368274 ,  0.30993764,  0.10963065, ...,  0.71434624,
        -0.33370432, -0.31942417],
       [-0.4368274 ,  0.3123805 ,  0.09706429, ...,  0.71434624,
        -0.33370432, -0.31942417]])

In [64]:
testdata = scaler.transform(testdata)
testdata

array([[-0.43967953,  2.77474712,  0.57115121, ..., -1.39988137,
        -0.33370432, -0.31942417],
       [-0.43967953,  1.14800547,  0.61303908, ...,  0.71434624,
        -0.33370432, -0.31942417],
       [-0.44007293,  0.2875366 ,  0.716921  , ...,  0.71434624,
        -0.33370432, -0.31942417],
       ...,
       [-0.43868262, -1.0482099 ,  1.65043408, ...,  0.71434624,
        -0.33370432, -0.31942417],
       [-0.43908496, -0.48906654,  0.67017413, ..., -1.39988137,
        -0.33370432, -0.31942417],
       [-0.43908496, -0.80032009,  0.62619187, ...,  0.71434624,
        -0.33370432, -0.31942417]])

In [65]:
traindata = pd.DataFrame(traindata)
testdata = pd.DataFrame(testdata)

In [66]:
x_train, x_test, y_train, y_test = ms.train_test_split(traindata, 
                                                       train.Item_Store_Returns, 
                                                       test_size=0.2, 
                                                       random_state=42)

In [67]:
x_train.shape, x_test.shape

((3992, 58), (998, 58))

In [69]:
traindata.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,-0.438191,0.030513,-0.287047,0.026132,-2.336230,0.742162,-0.990056,1.562491,1.155517,-1.359940,...,-0.367591,-0.681324,-0.613329,-0.645919,-0.681008,1.253860,2.427399,-1.399881,-0.333704,-0.319424
1,-0.438191,-0.490159,-0.301708,-1.301998,0.426800,0.742162,-0.990056,-0.640063,1.155517,0.714203,...,2.720418,-0.681324,-0.613329,-0.645919,-0.681008,1.253860,-0.411964,0.714346,-0.333704,-0.319424
2,-0.438191,-0.485151,-0.343596,1.112784,0.382943,0.742162,-0.990056,1.562491,-0.256331,0.714203,...,-0.367591,-0.681324,-0.613329,-0.645919,1.468411,-0.797537,-0.411964,0.714346,-0.333704,-0.319424
3,-0.438191,-0.486372,-0.307991,1.354262,0.397562,0.742162,-0.990056,-0.639984,1.155517,-1.431001,...,-0.367591,1.467731,-0.613329,-0.645919,-0.681008,1.253860,-0.411964,-1.399881,2.996665,-0.319424
4,-0.438191,-1.261307,-0.310086,0.750567,0.412181,-3.058250,-0.990056,-0.640001,-0.256331,0.714203,...,-0.367591,-0.681324,1.630447,-0.645919,1.468411,-0.797537,-0.411964,0.714346,-0.333704,-0.319424


In [86]:
cd

/home/olawale


In [87]:
cd Downloads

/home/olawale/Downloads


In [90]:
cat1 = pd.read_csv('baseline_cat (1).csv')
cat2 = pd.read_csv('maxdepth_8_cat.csv')

In [91]:
cat1_2 = (cat1.Item_Store_Returns + cat2.Item_Store_Returns)/2

In [92]:
cd

/home/olawale


In [93]:
pwd

'/home/olawale'

In [94]:
cd './Desktop/ZINDI-Competitions/ZINDI/DSN-Profit-Return-Challenge/'

/home/olawale/Desktop/ZINDI-Competitions/ZINDI/DSN-Profit-Return-Challenge


In [95]:
test = pd.read_csv('test.csv')

In [96]:
sample.Item_ID = test.Item_ID
sample.Item_Store_Returns = cat1_2

In [97]:
sample.to_csv('cat1_2.csv', index=False)